# Main ETL for Loan Level Data

## Initial set up

In [6]:
import glob
import sys
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import DateType, StringType, DoubleType, BooleanType, TimestampType
from functools import reduce
import pandas as pd

SPARK = SparkSession.builder.master("local[*]").getOrCreate()
BRONZE_SOURCE_DIR = "../data/mini_source"
SILVER_SOURCE_DIR = "../data/output/bronze"

### ASSET SILVER

In [19]:
def set_job_params():
    """
    Setup parameters used for this module.

    :return config: dictionary with properties used in this job.
    """
    config = {}
    config["SOURCE_DIR"] = SILVER_SOURCE_DIR
    config["DATE_COLUMNS"] = [
        "AS1",
        "AS19",
        "AS20",
        "AS31",
        "AS50",
        "AS51",
        "AS67",
        "AS70",
        "AS71",
        "AS87",
        "AS91",
        "AS112",
        "AS124",
        "AS127",
        "AS130",
        "AS133",
        "AS134",
        "AS137",
    ]
    config["ASSET_COLUMNS"] = {
        "AS1": DateType(),
        "AS2": StringType(),
        "AS3": StringType(),
        "AS4": StringType(),
        "AS5": StringType(),
        "AS6": StringType(),
        "AS7": StringType(),
        "AS8": StringType(),
        "AS15": StringType(),
        "AS16": StringType(),
        "AS17": StringType(),
        "AS18": StringType(),
        "AS19": DateType(),
        "AS20": DateType(),
        "AS21": StringType(),
        "AS22": StringType(),
        "AS23": BooleanType(),
        "AS24": StringType(),
        "AS25": StringType(),
        "AS26": StringType(),
        "AS27": DoubleType(),
        "AS28": DoubleType(),
        "AS29": BooleanType(),
        "AS30": DoubleType(),
        "AS31": DateType(),
        "AS32": StringType(),
        "AS33": StringType(),
        "AS34": StringType(),
        "AS35": StringType(),
        "AS36": StringType(),
        "AS37": DoubleType(),
        "AS38": DoubleType(),
        "AS39": DoubleType(),
        "AS40": DoubleType(),
        "AS41": DoubleType(),
        "AS42": StringType(),
        "AS43": StringType(),
        "AS44": DoubleType(),
        "AS45": StringType(),
        "AS50": DateType(),
        "AS51": DateType(),
        "AS52": StringType(),
        "AS53": BooleanType(),
        "AS54": DoubleType(),
        "AS55": DoubleType(),
        "AS56": DoubleType(),
        "AS57": StringType(),
        "AS58": StringType(),
        "AS59": StringType(),
        "AS60": DoubleType(),
        "AS61": DoubleType(),
        "AS62": StringType(),
        "AS63": DoubleType(),
        "AS64": DoubleType(),
        "AS65": StringType(),
        "AS66": DoubleType(),
        "AS67": DateType(),
        "AS68": StringType(),
        "AS69": DoubleType(),
        "AS70": DateType(),
        "AS71": DateType(),
        "AS80": DoubleType(),
        "AS81": DoubleType(),
        "AS82": DoubleType(),
        "AS83": StringType(),
        "AS84": StringType(),
        "AS85": DoubleType(),
        "AS86": DoubleType(),
        "AS87": DateType(),
        "AS88": DoubleType(),
        "AS89": StringType(),
        "AS90": DoubleType(),
        "AS91": DateType(),
        "AS92": StringType(),
        "AS93": DoubleType(),
        "AS94": StringType(),
        "AS100": DoubleType(),
        "AS101": DoubleType(),
        "AS102": DoubleType(),
        "AS103": DoubleType(),
        "AS104": DoubleType(),
        "AS105": DoubleType(),
        "AS106": DoubleType(),
        "AS107": DoubleType(),
        "AS108": DoubleType(),
        "AS109": DoubleType(),
        "AS110": DoubleType(),
        "AS111": StringType(),
        "AS112": DateType(),
        "AS115": DoubleType(),
        "AS116": DoubleType(),
        "AS117": DoubleType(),
        "AS118": DoubleType(),
        "AS119": DoubleType(),
        "AS120": DoubleType(),
        "AS121": BooleanType(),
        "AS122": BooleanType(),
        "AS123": StringType(),
        "AS124": DateType(),
        "AS125": DoubleType(),
        "AS126": DoubleType(),
        "AS127": DateType(),
        "AS128": DoubleType(),
        "AS129": StringType(),
        "AS130": DateType(),
        "AS131": BooleanType(),
        "AS132": DoubleType(),
        "AS133": DateType(),
        "AS134": DateType(),
        "AS135": DoubleType(),
        "AS136": DoubleType(),
        "AS137": DateType(),
        "AS138": DoubleType(),
    }
    return config

def replace_no_data(df):
    """
    Replace ND values inside the dataframe
    TODO: ND are associated with labels that explain why the vaue is missing.
          Should handle this information better in future releases.
    :param df: Spark dataframe with loan asset data.
    :return df: Spark dataframe without ND values.
    """
    for col_name in df.columns:
        df = df.withColumn(
            col_name,
            F.when(F.col(col_name).startswith("ND"), None).otherwise(F.col(col_name)),
        )
    return df


def replace_bool_data(df):
    """
    Replace Y/N with boolean flags in the dataframe.

    :param df: Spark dataframe with loan asset data.
    :return df: Spark dataframe without Y/N values.
    """
    for col_name in df.columns:
        df = df.withColumn(
            col_name,
            F.when(F.col(col_name) == "Y", "True")
            .when(F.col(col_name) == "N", "False")
            .otherwise(F.col(col_name)),
        )
    return df


def cast_to_datatype(df, columns):
    """
    Cast data to the respective datatype.

    :param df: Spark dataframe with loan asset data.
    :param columns: collection of column names and respective data types.
    :return df: Spark dataframe with correct values.
    """
    for col_name, data_type in columns.items():
        if data_type == BooleanType():
            df = (
                df.withColumn(col_name, F.col(col_name).contains("True"))
            )
        if data_type == DateType():
            df = (
                df.withColumn(col_name, F.to_date(F.col(col_name)))
            )
        if data_type == DoubleType():
            df = (
                df.withColumn(
                    col_name, F.round(F.col(col_name).cast(DoubleType()), 2)
                )
            )
    return df

def get_columns_collection(df):
    """
    Get collection of dataframe columns divided by topic.

    :param df: Asset bronze Spark dataframe.
    :return cols_dict: collection of columns labelled by topic.
    """
    cols_dict = {
        "general": ["ed_code","valid_from", "valid_to", "iscurrent", "checksum", "year", "month"]
        + [f"AS{i}" for i in range(1, 15) if f"AS{i}" in df.columns],
        "obligor_info": [f"AS{i}" for i in range(15, 50) if f"AS{i}" in df.columns],
        "loan_info": [f"AS{i}" for i in range(50, 80) if f"AS{i}" in df.columns],
        "interest_rate": [f"AS{i}" for i in range(80, 100) if f"AS{i}" in df.columns],
        "financial_info": [f"AS{i}" for i in range(100, 115) if f"AS{i}" in df.columns],
        "performance_info": [
            f"AS{i}" for i in range(115, 146) if f"AS{i}" in df.columns
        ],
    }
    return cols_dict


def process_dates(df, date_cols_list):
    """
    Extract dates dimension from bronze Spark dataframe.

    :param df: Spark bronze dataframe.
    :param date_cols_list: list of date columns.
    :return new_df: silver type Spark dataframe.
    """
    date_cols = [c for c in date_cols_list if c in df.columns]

    new_df = (
        df.select(F.explode(F.array(date_cols)).alias("date_col"))
        .dropDuplicates()
        .withColumn("unix_date", F.unix_timestamp(F.col("date_col")))
        .withColumn("year", F.year(F.col("date_col")))
        .withColumn("month", F.month(F.col("date_col")))
        .withColumn("quarter", F.quarter(F.col("date_col")))
        .withColumn("WoY", F.weekofyear(F.col("date_col")))
        .withColumn("day", F.dayofmonth(F.col("date_col")))
    )
    return new_df


def process_obligor_info(df, cols_dict):
    """
    Extract obligor info dimension from bronze Spark dataframe.

    :param df: Spark bronze dataframe.
    :param cols_dict: collection of columns labelled by their topic.
    :return new_df: silver type Spark dataframe.
    """
    new_df = (
        df.select(cols_dict["general"] + cols_dict["obligor_info"])
        .withColumn("AS1", F.unix_timestamp(F.col("AS1")))
        .withColumn("AS19", F.unix_timestamp(F.col("AS19")))
        .withColumn("AS20", F.unix_timestamp(F.col("AS20")))
        .withColumn("AS31", F.unix_timestamp(F.col("AS31")))
    )
    return new_df


def process_loan_info(df, cols_dict):
    """
    Extract loan info dimension from bronze Spark dataframe.

    :param df: Spark bronze dataframe.
    :param cols_dict: collection of columns labelled by their topic.
    :return new_df: silver type Spark dataframe.
    """
    new_df = (
        df.select(cols_dict["general"] + cols_dict["loan_info"])
        .withColumn("AS1", F.unix_timestamp(F.col("AS1")))
        .withColumn("AS50", F.unix_timestamp(F.col("AS50")))
        .withColumn("AS51", F.unix_timestamp(F.col("AS51")))
        .withColumn("AS67", F.unix_timestamp(F.col("AS67")))
        .withColumn("AS70", F.unix_timestamp(F.col("AS70")))
        .withColumn("AS71", F.unix_timestamp(F.col("AS71")))
    )
    return new_df


def process_interest_rate(df, cols_dict):
    """
    Extract interest rate dimension from bronze Spark dataframe.

    :param df: Spark bronze dataframe.
    :param cols_dict: collection of columns labelled by their topic.
    :return new_df: silver type Spark dataframe.
    """
    new_df = (
        df.select(cols_dict["general"] + cols_dict["interest_rate"])
        .withColumn("AS1", F.unix_timestamp(F.col("AS1")))
        .withColumn("AS87", F.unix_timestamp(F.col("AS87")))
        .withColumn("AS91", F.unix_timestamp(F.col("AS91")))
    )
    return new_df


def process_financial_info(df, cols_dict):
    """
    Extract financial info dimension from bronze Spark dataframe.

    :param df: Spark bronze dataframe.
    :param cols_dict: collection of columns labelled by their topic.
    :return new_df: silver type Spark dataframe.
    """
    new_df = (
        df.select(cols_dict["general"] + cols_dict["financial_info"])
        .withColumn("AS1", F.unix_timestamp(F.col("AS1")))
        .withColumn("AS112", F.unix_timestamp(F.col("AS112")))
    )
    return new_df


def process_performance_info(df, cols_dict):
    """
    Extract performance info dimension from bronze Spark dataframe.

    :param df: Spark bronze dataframe.
    :param cols_dict: collection of columns labelled by their topic.
    :return new_df: silver type Spark dataframe.
    """
    new_df = (
        df.select(cols_dict["general"] + cols_dict["performance_info"])
        .withColumn("AS1", F.unix_timestamp(F.col("AS1")))
        .withColumn("AS124", F.unix_timestamp(F.col("AS124")))
        .withColumn("AS127", F.unix_timestamp(F.col("AS127")))
        .withColumn("AS130", F.unix_timestamp(F.col("AS130")))
        .withColumn("AS133", F.unix_timestamp(F.col("AS133")))
        .withColumn("AS134", F.unix_timestamp(F.col("AS134")))
        .withColumn("AS137", F.unix_timestamp(F.col("AS137")))
    )
    return new_df


run_props = set_job_params()
bronze_df = SPARK.read.parquet(
    f'{run_props["SOURCE_DIR"]}/assets.parquet'
)
assets_columns = get_columns_collection(bronze_df)
print("Remove ND values.")
tmp_df1 = replace_no_data(bronze_df)
print("Replace Y/N with boolean flags.")
tmp_df2 = replace_bool_data(tmp_df1)
print("Cast data to correct types.")
cleaned_df = cast_to_datatype(tmp_df2, run_props["ASSET_COLUMNS"])
print("Generate time dataframe")
date_df = process_dates(cleaned_df, run_props["DATE_COLUMNS"])
print("Generate obligor info dataframe")
obligor_info_df = process_obligor_info(cleaned_df, assets_columns)
print("Generate loan info dataframe")
loan_info_df = process_loan_info(cleaned_df, assets_columns)
print("Generate interest rate dataframe")
interest_rate_df = process_interest_rate(cleaned_df, assets_columns)
print("Generate financial info dataframe")
financial_info_df = process_financial_info(cleaned_df, assets_columns)
print("Generate performace info dataframe")
performance_info_df = process_performance_info(cleaned_df, assets_columns)

print("Write dataframe")

(
    date_df.write
    .mode("overwrite")
    .parquet("../data/output/silver/assets/date_table.parquet")
)
(
    loan_info_df.write
    .partitionBy("year", "month")
    .mode("overwrite")
    .parquet("../data/output/silver/assets/loan_info_table.parquet")
)
(
    obligor_info_df.write
    .partitionBy("year", "month")
    .mode("overwrite")
    .parquet("../data/output/silver/assets/obligor_info_table.parquet")
)
(
    financial_info_df.write
    .partitionBy("year", "month")
    .mode("overwrite")
    .parquet("../data/output/silver/assets/financial_info_table.parquet")
)
(
    interest_rate_df.write
    .partitionBy("year", "month")
    .mode("overwrite")
    .parquet("../data/output/silver/assets/interest_rate_table.parquet")
)
(
    performance_info_df.write
    .partitionBy("year", "month")
    .mode("overwrite")
    .parquet("../data/output/silver/assets/performance_info_table.parquet")
)

Remove ND values.
Replace Y/N with boolean flags.
Cast data to correct types.
Generate time dataframe
Generate obligor info dataframe
Generate loan info dataframe
Generate interest rate dataframe
Generate financial info dataframe
Generate performace info dataframe
Write dataframe


### AMORTISATION SILVER

In [35]:
def set_job_params():
    """
    Setup parameters used for this module.

    :return config: dictionary with properties used in this job.
    """
    config = {}
    config["SOURCE_DIR"] = SILVER_SOURCE_DIR
    config["AMORTISATION_COLUMNS"] = {
        "AS3": StringType(),
        "AS150": DoubleType(),
        "AS151": DateType(),
        "AS1348": DoubleType(),
        "AS1349": DateType(),
    }
    for i in range(152, 1348):
        if i % 2 == 0:
            config["AMORTISATION_COLUMNS"][f"AS{i}"] = DoubleType()
        else:
            config["AMORTISATION_COLUMNS"][f"AS{i}"] = DateType()
    return config


def _melt(df, id_vars, value_vars, var_name="FEATURE_NAME", value_name="FEATURE_VALUE"):
    """Convert DataFrame from wide to long format."""
    # Ref:https://stackoverflow.com/a/41673644
    # Create array<struct<variable: str, value: ...>>
    _vars_and_vals = F.array(
        *(
            F.struct(F.lit(c).alias(var_name), F.col(c).alias(value_name))
            for c in value_vars
        )
    )
    # Add to the DataFrame and explode
    _tmp = df.withColumn("_vars_and_vals", F.explode(_vars_and_vals))
    cols = id_vars + [
        F.col("_vars_and_vals")[x].cast("string").alias(x)
        for x in [var_name, value_name]
    ]
    return _tmp.select(*cols)


def unpivot_dataframe(df, columns):
    """
    Convert dataframe from wide to long table.

    :param df: raw Spark dataframe.
    :param columns: data columns with respective datatype.
    :return new_df: unpivot Spark dataframe.
    """
    df = df.withColumn("AS3", F.concat_ws("_",F.col("AS3"), F.monotonically_increasing_id()))
    date_columns = [
        k for k, v in columns.items() if v == DateType() and k in df.columns
    ]
    double_columns = [
        k for k, v in columns.items() if v == DoubleType() and k in df.columns
    ]

    date_df = (
            _melt(
                df,
                id_vars=["AS3"],
                value_vars=date_columns,
                var_name="DATE_COLUMNS",
                value_name="DATE_VALUE",
                )
            .filter(F.col("DATE_VALUE").isNotNull())
        )
    double_df = (
            _melt(
                df,
                id_vars=["AS3"],
                value_vars=double_columns,
                var_name="DOUBLE_COLUMNS",
                value_name="DOUBLE_VALUE",
                )
        )
    scd2_df = df.select("AS3","ed_code","valid_from", "valid_to", "iscurrent", "checksum", "year", "month")
    new_df = (  
        date_df
        .join(double_df, on="AS3", how="inner")
        .join(scd2_df, on="AS3", how="inner")
        .withColumn("AS3", F.split(F.col("AS3"),"_").getItem(0))
    )
    return new_df

def cast_to_datatype(df, columns):
    """
    Cast data to the respective datatype.

    :param df: Spark dataframe with loan asset data.
    :param columns: collection of column names and respective data types.
    :return df: Spark dataframe with correct values.
    """
    for col_name, data_type in columns.items():
        if data_type == DateType():
            df = df.withColumn(col_name, F.to_date(F.col(col_name)))
        if data_type == DoubleType():
            df = df.withColumn(col_name, F.round(F.col(col_name).cast(DoubleType()), 2))
    return df

def process_dates(df):
    """
    Extract dates dimension from bronze Spark dataframe.

    :param df: Spark bronze dataframe.
    :return new_df: silver type Spark dataframe.
    """
    new_df = (
        df.select("DATE_VALUE")
        .alias("date_col")
        .dropDuplicates()
        .withColumn("unix_date", F.unix_timestamp(F.col("date_col")))
        .withColumn("year", F.year(F.col("date_col")))
        .withColumn("month", F.month(F.col("date_col")))
        .withColumn("quarter", F.quarter(F.col("date_col")))
        .withColumn("WoY", F.weekofyear(F.col("date_col")))
        .withColumn("day", F.dayofmonth(F.col("date_col")))
    )
    return new_df

def process_info(df):
    """
    Extract amortisation values dimension from bronze Spark dataframe.

    :param df: Spark bronze dataframe.
    :return new_df: silver type Spark dataframe.
    """
    new_df = df.withColumn(
        "tmp_DATE_VALUE", F.unix_timestamp(F.col("DATE_VALUE"))
    ).drop("DATE_VALUE")
    return new_df


print("Start AMORTISATION SILVER job.")
run_props = set_job_params()
bronze_df = SPARK.read.parquet(
    f'{run_props["SOURCE_DIR"]}/amortisation.parquet'
)
print("Cast data to correct types.")
tmp_df1 = unpivot_dataframe(bronze_df, run_props["AMORTISATION_COLUMNS"])
cleaned_df = (
    tmp_df1.withColumn("DATE_VALUE", F.to_date(F.col("DATE_VALUE")))
    .withColumn(
        "DOUBLE_VALUE", F.round(F.col("DOUBLE_VALUE").cast(DoubleType()), 2)
    )
)
# print("Generate time dataframe")
# date_df = process_dates(bronze_df, run_props["DATE_COLUMNS"])
# print("Generate info dataframe")
# info_df = process_info(bronze_df)
# print("Write dataframe")

# (
#     date_df.write
#     .mode("overwrite")
#     .parquet("../data/output/silver/amortisation/date_table.parquet")
# )
# (
#     info_df.write
#     .partitionBy("year", "month", "day")
#     .mode("overwrite")
#     .parquet("../data/output/silver/amortisation/info_table.parquet")
# )


Start AMORTISATION SILVER job.
Cast data to correct types.


In [36]:
cleaned_df.show(n=100)

+---------------+------------+----------+--------------+------------+--------------------+--------------------+--------+---------+--------------------+----+-----+
|            AS3|DATE_COLUMNS|DATE_VALUE|DOUBLE_COLUMNS|DOUBLE_VALUE|             ed_code|          valid_from|valid_to|iscurrent|            checksum|year|month|
+---------------+------------+----------+--------------+------------+--------------------+--------------------+--------+---------+--------------------+----+-----+
|000807376530886|       AS151|2013-01-05|        AS1346|         0.0|SMESES00017610032...|2022-11-19 16:03:...|    null|        1|2f5ad00e06a0ec78d...|2012|   12|
|000807376530886|       AS151|2013-01-05|        AS1344|         0.0|SMESES00017610032...|2022-11-19 16:03:...|    null|        1|2f5ad00e06a0ec78d...|2012|   12|
|000807376530886|       AS151|2013-01-05|        AS1342|         0.0|SMESES00017610032...|2022-11-19 16:03:...|    null|        1|2f5ad00e06a0ec78d...|2012|   12|
|000807376530886|     

In [34]:
data = [
    ["1","2022-01-01",0.1,"2022-01-01",0.5,"r1"],
    ["2","2022-02-01",0.2,"2022-02-01",0.6,"r2"],
    ["3","2022-03-01",0.3,"2022-03-01",0.7,"r3"],
    ["4","2022-04-01",0.4,None,0.8,"r4"],
]
col = ["AS3","t1","v1","t2","v2","r_id"]


test = SPARK.createDataFrame(data, col).withColumn("AS3", F.concat_ws("_",F.col("AS3"), F.monotonically_increasing_id()))
date_df = (
        _melt(
            test,
            id_vars=["AS3"],
            value_vars=["t1","t2"],
            var_name="DATE_COLUMNS",
            value_name="DATE_VALUE",
            )
        .filter(F.col("DATE_VALUE").isNotNull())
    )
double_df = (
        _melt(
            test,
            id_vars=["AS3"],
            value_vars=["v1","v2"],
            var_name="DOUBLE_COLUMNS",
            value_name="DOUBLE_VALUE",
            )
    )
scd2_df = test.select("AS3","r_id")
new_df = (
        date_df
        .join(double_df, on="AS3", how="inner")
        .join(scd2_df, on="AS3", how="inner")
        .withColumn("AS3", F.split(F.col("AS3"),"_").getItem(0))
    )
new_df.show()

+---+------------+----------+--------------+------------+----+
|AS3|DATE_COLUMNS|DATE_VALUE|DOUBLE_COLUMNS|DOUBLE_VALUE|r_id|
+---+------------+----------+--------------+------------+----+
|  1|          t2|2022-01-01|            v2|         0.5|  r1|
|  1|          t2|2022-01-01|            v1|         0.1|  r1|
|  1|          t1|2022-01-01|            v2|         0.5|  r1|
|  1|          t1|2022-01-01|            v1|         0.1|  r1|
|  2|          t2|2022-02-01|            v2|         0.6|  r2|
|  2|          t2|2022-02-01|            v1|         0.2|  r2|
|  2|          t1|2022-02-01|            v2|         0.6|  r2|
|  2|          t1|2022-02-01|            v1|         0.2|  r2|
|  3|          t2|2022-03-01|            v2|         0.7|  r3|
|  3|          t2|2022-03-01|            v1|         0.3|  r3|
|  3|          t1|2022-03-01|            v2|         0.7|  r3|
|  3|          t1|2022-03-01|            v1|         0.3|  r3|
|  4|          t1|2022-04-01|            v2|         0.

root
 |-- AS3: string (nullable = true)
 |-- AS150: string (nullable = true)
 |-- AS151: string (nullable = true)
 |-- AS152: string (nullable = true)
 |-- AS153: string (nullable = true)
 |-- AS154: string (nullable = true)
 |-- AS155: string (nullable = true)
 |-- AS156: string (nullable = true)
 |-- AS157: string (nullable = true)
 |-- AS158: string (nullable = true)
 |-- AS159: string (nullable = true)
 |-- AS160: string (nullable = true)
 |-- AS161: string (nullable = true)
 |-- AS162: string (nullable = true)
 |-- AS163: string (nullable = true)
 |-- AS164: string (nullable = true)
 |-- AS165: string (nullable = true)
 |-- AS166: string (nullable = true)
 |-- AS167: string (nullable = true)
 |-- AS168: string (nullable = true)
 |-- AS169: string (nullable = true)
 |-- AS170: string (nullable = true)
 |-- AS171: string (nullable = true)
 |-- AS172: string (nullable = true)
 |-- AS173: string (nullable = true)
 |-- AS174: string (nullable = true)
 |-- AS175: string (nullable = true

### BOND INFO SILVER

In [ ]:
def set_job_params():
    """
    Setup parameters used for this module.

    :return config: dictionary with properties used in this job.
    """
    config = {}
    config["SOURCE_DIR"] = SILVER_SOURCE_DIR
    config["DATE_COLUMNS"] = ["BS1"]
    return config


def get_columns_collection(df):
    """
    Get collection of dataframe columns divided by topic.

    :param df: Asset bronze Spark dataframe.
    :return cols_dict: collection of columns labelled by topic.
    """
    cols_dict = {
        "bond_info": ["ID", "year", "month", "day"]
        + [f"BS{i}" for i in range(1, 11) if f"BS{i}" in df.columns],
        "collateral_info": ["ID", "year", "month", "day"]
        + [f"BS{i}" for i in range(11, 19) if f"BS{i}" in df.columns],
        "contact_info": ["ID", "year", "month", "day"]
        + [f"BS{i}" for i in range(19, 25) if f"BS{i}" in df.columns],
    }
    return cols_dict


def process_dates(df, date_cols_list):
    """
    Extract dates dimension from bronze Spark dataframe.

    :param df: Spark bronze dataframe.
    :param date_cols_list: list of date columns.
    :return new_df: silver type Spark dataframe.
    """
    date_cols = [c for c in date_cols_list if c in df.columns]

    new_df = (
        df.select(F.explode(F.array(date_cols)).alias("date_col"))
        .dropDuplicates()
        .withColumn("unix_date", F.unix_timestamp(F.col("date_col")))
        .withColumn("year", F.year(F.col("date_col")))
        .withColumn("month", F.month(F.col("date_col")))
        .withColumn("quarter", F.quarter(F.col("date_col")))
        .withColumn("WoY", F.weekofyear(F.col("date_col")))
        .withColumn("day", F.dayofmonth(F.col("date_col")))
    )
    return new_df


def process_bond_info(df, cols_dict):
    """
    Extract bond info dimension from bronze Spark dataframe.

    :param df: Spark bronze dataframe.
    :param cols_dict: collection of columns labelled by their topic.
    :return new_df: silver type Spark dataframe.
    """
    new_df = (
        df.select(cols_dict["bond_info"])
        .withColumn(
            "tmp_BS1", F.unix_timestamp(F.to_timestamp(F.col("BS1"), "yyyy-MM-dd"))
        )
        .drop("BS1")
        .withColumnRenamed("tmp_BS1", "BS1")
    )
    return new_df


def process_collateral_info(df, cols_dict):
    """
    Extract collateral info dimension from bronze Spark dataframe.

    :param df: Spark bronze dataframe.
    :param cols_dict: collection of columns labelled by their topic.
    :return new_df: silver type Spark dataframe.
    """
    new_df = (
        df.select(["BS1", "BS2"] + cols_dict["collateral_info"])
        .withColumn(
            "tmp_BS1", F.unix_timestamp(F.to_timestamp(F.col("BS1"), "yyyy-MM-dd"))
        )
        .drop("BS1")
        .withColumnRenamed("tmp_BS1", "BS1")
    )
    return new_df


def process_contact_info(df, cols_dict):
    """
    Extract contact info dimension from bronze Spark dataframe.

    :param df: Spark bronze dataframe.
    :param cols_dict: collection of columns labelled by their topic.
    :return new_df: silver type Spark dataframe.
    """
    new_df = (
        df.select(["BS1", "BS2"] + cols_dict["contact_info"])
        .withColumn(
            "tmp_BS1", F.unix_timestamp(F.to_timestamp(F.col("BS1"), "yyyy-MM-dd"))
        )
        .drop("BS1")
        .withColumnRenamed("tmp_BS1", "BS1")
    )
    return new_df


print("Start BOND INFO SILVER job.")
run_props = set_job_params()
bronze_df = SPARK.read.parquet(
    f'{run_props["SOURCE_DIR"]}/bronze/bond_info.parquet'
)
bond_info_columns = get_columns_collection(bronze_df)
print("Generate time dataframe")
date_df = process_dates(bronze_df, run_props["DATE_COLUMNS"])
print("Generate bond info dataframe")
info_df = process_bond_info(bronze_df, bond_info_columns)
print("Generate collateral info dataframe")
collateral_df = process_collateral_info(bronze_df, bond_info_columns)
print("Generate contact info dataframe")
contact_df = process_contact_info(bronze_df, bond_info_columns)

print("Write dataframe")

(
    date_df.write
    .mode("overwrite")
    .parquet("../data/output/silver/bond_info/date_table.parquet")
)
(
    info_df.write
    .partitionBy("year", "month", "day")
    .mode("overwrite")
    .parquet("../data/output/silver/bond_info/info_table.parquet")
)
(
    collateral_df.write
    .partitionBy("year", "month", "day")
    .mode("overwrite")
    .parquet("../data/output/silver/bond_info/collaterals_table.parquet")
)
(
    contact_df.write
    .partitionBy("year", "month", "day")
    .mode("overwrite")
    .parquet("../data/output/silver/bond_info/contacts_table.parquet")
)


### COLLATERAL SILVER

In [ ]:
def set_job_params():
    """
    Setup parameters used for this module.

    :return config: dictionary with properties used in this job.
    """
    config = {}
    config["SOURCE_DIR"] = SILVER_SOURCE_DIR
    config["DATE_COLUMNS"] = ["CS11", "CS12", "CS22"]
    return config


def process_dates(df, date_cols_list):
    """
    Extract dates dimension from bronze Spark dataframe.

    :param df: Spark bronze dataframe.
    :param date_cols_list: list of date columns.
    :return new_df: silver type Spark dataframe.
    """
    date_cols = [c for c in date_cols_list if c in df.columns]

    new_df = (
        df.select(F.explode(F.array(date_cols)).alias("date_col"))
        .dropDuplicates()
        .withColumn("unix_date", F.unix_timestamp(F.col("date_col")))
        .withColumn("year", F.year(F.col("date_col")))
        .withColumn("month", F.month(F.col("date_col")))
        .withColumn("quarter", F.quarter(F.col("date_col")))
        .withColumn("WoY", F.weekofyear(F.col("date_col")))
        .withColumn("day", F.dayofmonth(F.col("date_col")))
    )
    return new_df


def process_collateral_info(df):
    """
    Extract collateral info dimension from bronze Spark dataframe.

    :param df: Spark bronze dataframe.
    :return new_df: silver type Spark dataframe.
    """
    new_df = (
        df.withColumn(
            "tmp_CS11", F.unix_timestamp(F.to_timestamp(F.col("CS11"), "yyyy-MM"))
        )
        .drop("CS11")
        .withColumnRenamed("tmp_CS11", "CS11")
        .withColumn(
            "tmp_CS12", F.unix_timestamp(F.to_timestamp(F.col("CS12"), "yyyy-MM"))
        )
        .drop("CS12")
        .withColumnRenamed("tmp_CS12", "CS12")
        .withColumn(
            "tmp_CS22", F.unix_timestamp(F.to_timestamp(F.col("CS22"), "yyyy-MM"))
        )
        .drop("CS22")
        .withColumnRenamed("tmp_CS22", "CS22")
    )
    return new_df


print("Start COLLATERAL SILVER job.")
run_props = set_job_params()
bronze_df = SPARK.read.parquet(
    f'{run_props["SOURCE_DIR"]}/bronze/collaterals.parquet'
)
print("Generate collateral info dataframe")
info_df = process_collateral_info(bronze_df)
print("Generate time dataframe")
date_df = process_dates(bronze_df, run_props["DATE_COLUMNS"])

print("Write dataframe")

(
    info_df.write
    .partitionBy("year", "month", "day")
    .mode("overwrite")
    .parquet("../data/output/silver/collaterals/info_table.parquet")
)
(
    date_df.write
    .mode("overwrite")
    .parquet("../data/output/silver/collaterals/date_table.parquet")
)